In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [3]:


class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            super().__init__()
            self.K = param_K  # パラメターKを設定する。
            self.indicator = {}

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean() * self.K     #　過去５本足のレンジの平均値 * K

        # Long/Short判定
        diff_low   =   (ohlcv_latest["open"][0] - ohlcv_latest["low"][0])
        diff_high  = - (ohlcv_latest["open"][0] - ohlcv_latest["high"][0])  

        detect_short  = range_width < diff_low
        detect_long = range_width  < diff_high
        
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range', time_us, range_width)
        
        if detect_short and detect_long:
            print("detect both side")
            if diff_low < diff_high:
                detect_long = True
                detect_short = False
            else:
                detect_long = False
                detect_short = True

        #print(range_width*self.K, diff_low, diff_high, detect_long, detect_short)

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.place_order("Buy", session.best_buy_price+0.1, 0.01, 600, "Open Long")    
                else:

                    session.place_order("Buy", session.best_buy_price+0.1, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.place_order("Sell", session.best_sell_price-0.1, 0.01, 600, "Open Short") 
                else:
                    session.place_order("Sell", session.best_sell_price-0.1, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [5]:
agent = Agent()

print("start")
r = back_tester.run(agent, rbot.DAYS_BEFORE(20), 0)
#r = back_tester.run(Agent(), 0, 0)
print("end")




start
detect both side
end


In [6]:
import rbot
df = rbot.result_to_df(r)

In [7]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-11-22 14:00:00+00:00,2022-11-22 14:10:00.006000+00:00,0000-0001,0,Buy,True,Expire,0.0,0.00,0.00,0.0,...,15730.10,0.01,157.3010,0.0,0.00000,0.00000,0.00,Open Long,0.00000,0.00
2022-11-23 04:00:00+00:00,2022-11-23 04:10:00.023000+00:00,0000-0002,0,Buy,True,Expire,0.0,0.00,0.00,0.0,...,16201.73,0.01,162.0173,0.0,0.00000,0.00000,0.00,Open Long,0.00000,0.00
2022-11-23 20:00:00+00:00,2022-11-23 20:10:00.097000+00:00,0000-0003,0,Buy,True,Expire,0.0,0.00,0.00,0.0,...,16373.15,0.01,163.7315,0.0,0.00000,0.00000,0.00,Open Long,0.00000,0.00
2022-11-25 06:00:00+00:00,2022-11-25 06:10:00.029000+00:00,0000-0004,0,Sell,True,Expire,0.0,0.00,0.00,0.0,...,16474.71,0.01,164.7471,0.0,0.00000,0.00000,0.00,Open Short,0.00000,0.00
2022-11-26 02:00:00+00:00,2022-11-26 02:10:00.096000+00:00,0000-0005,0,Buy,True,Expire,0.0,0.00,0.00,0.0,...,16514.05,0.01,165.1405,0.0,0.00000,0.00000,0.00,Open Long,0.00000,0.00
2022-11-26 16:00:00+00:00,2022-11-26 16:10:00.024000+00:00,0000-0006,0,Sell,True,Expire,0.0,0.00,0.00,0.0,...,16588.55,0.01,165.8855,0.0,0.00000,0.00000,0.00,Open Short,0.00000,0.00
2022-11-28 00:00:00+00:00,2022-11-28 00:10:00.041000+00:00,0000-0007,0,Sell,True,Expire,0.0,0.00,0.00,0.0,...,16569.37,0.01,165.6937,0.0,0.00000,0.00000,0.00,Open Short,0.00000,0.00
2022-11-28 02:00:00+00:00,2022-11-28 02:10:00.095000+00:00,0000-0008,0,Sell,True,Expire,0.0,0.00,0.00,0.0,...,16422.33,0.01,164.2233,0.0,0.00000,0.00000,0.00,Open Short,0.00000,0.00
2022-11-28 18:00:00+00:00,2022-11-28 18:07:22.655000+00:00,0000-0009,0,Buy,True,Open,16141.0,0.01,161.41,0.0,...,16141.00,0.01,161.4100,0.0,0.16141,-0.16141,0.01,Open Long,-0.16141,0.01


In [8]:
market = Market.open("BN", "BTCBUSD")
ohlcv = market.ohlcv(rbot.NOW()-rbot.DAYS(20), 0, 60*60)


In [9]:
from collections import OrderedDict
import numpy as np


from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import datetime




In [10]:
chart = Chart(900, 400, ohlcv)
p = chart.new_figure('indicator', 200, 'indicator')

indicator = agent.get_indicator('diff_low')
chart.line(p, indicator, 'value', 'diff_low', color='#ff8080')

indicator = agent.get_indicator('diff_high')
chart.line(p, indicator, 'value', 'diff_high', color='#80ff80')

indicator = agent.get_indicator('range')
chart.line(p, indicator, 'value', 'range', color='#8080ff')

chart.show()

Loading BokehJS ...

price
volume
indicator


Loading BokehJS ...

main


In [10]:
from bokeh.models.glyphs import Step
from bokeh.models import ColumnDataSource

chart2 = Chart(800, 400, ohlcv)
p = chart2.new_figure('profit', 100, 'profit')
p.step(x=df['update_time'], y=df['sum_profit'])

p = chart2.get_figure('main')
buy_df = df[df['order_side'] == "Buy"]
p.scatter(x=buy_df['update_time'], y=buy_df['order_price'], marker='triangle', size=10, color="#6699ff")

sell_df = df[df['order_side'] == "Sell"]
p.scatter(x=sell_df.index, y=sell_df['order_price'], marker='inverted_triangle', size=10, color="#ff8080")


p = chart2.new_figure('indicator', 150, 'indicator')

indicator = agent.get_indicator('diff_low')
p.line(x=indicator.index, y=indicator['value'], color='#ff8080')

indicator = agent.get_indicator('diff_high')
p.line(x=indicator.index, y=indicator['value'], color='#80ff80')

indicator = agent.get_indicator('range')
p.line(x=indicator.index, y=indicator['value'], color='#8080ff')


chart2.show()

Loading BokehJS ...

main
profit
indicator


In [ ]:
agent.get_indicator_names()

['diff_low', 'diff_high', 'range']

In [ ]:
indicator

,diff_low
timestamp,
NaT,"[1669910400000000, 194.09000000000015]"
NaT,"[1669917600000000, 58.11999999999898]"
NaT,"[1669924800000000, 68.33000000000175]"
NaT,"[1669932000000000, 106.88000000000102]"
NaT,"[1669939200000000, 3.2400000000016007]"
...,...
NaT,"[1670601600000000, 70.20000000000073]"
NaT,"[1670608800000000, 30.340000000000146]"
NaT,"[1670616000000000, 44.919999999998254]"


In [11]:
agent.get_indicator('diff_low')

ValueError: Must pass 2-d input. shape=()

In [19]:
buy_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-04 02:00:00+00:00,2022-12-04 02:10:00.004000+00:00,0000-0003,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,16888.07,0.01,168.8807,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-05 02:00:00+00:00,2022-12-05 02:10:00.152000+00:00,0000-0004,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,17109.65,0.01,171.0965,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-05 06:00:00+00:00,2022-12-05 06:10:00.193000+00:00,0000-0005,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,17200.68,0.01,172.0068,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-08 20:00:00+00:00,2022-12-08 20:10:00.102000+00:00,0000-0008,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,16967.98,0.01,169.6798,0.0,0.0,0.0,0.0,Open Long,0.0,0.0


In [20]:
sell_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-02 14:00:00+00:00,2022-12-02 14:10:00.053000+00:00,0000-0001,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,16997.89,0.01,169.9789,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-04 00:00:00+00:00,2022-12-04 00:10:00.020000+00:00,0000-0002,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,16949.21,0.01,169.4921,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-05 16:00:00+00:00,2022-12-05 16:10:00.066000+00:00,0000-0006,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17234.33,0.01,172.3433,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-07 08:00:00+00:00,2022-12-07 08:10:00.030000+00:00,0000-0007,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17005.57,0.01,170.0557,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-09 14:00:00+00:00,2022-12-09 14:10:00.314000+00:00,0000-0009,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17242.62,0.01,172.4262,0.0,0.0,0.0,0.0,Open Short,0.0,0.0


In [21]:
rbot.init_debug_log()
bn.download(10)

NameError: name 'bn' is not defined

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time

In [ ]:
df['order_side'] == rbot.OrderSide.Buy

create_time
2022-12-01 14:00:00+00:00    False
2022-12-02 14:00:00+00:00     True
2022-12-04 02:00:00+00:00    False
2022-12-04 02:00:00+00:00    False
2022-12-05 16:00:00+00:00     True
2022-12-05 16:00:00+00:00     True
Name: order_side, dtype: bool

In [68]:
indicators = {}

indicators['a'] = []

indicators['a'].append((1, 1.0))
indicators['a'].append((2, 2.0))
    

In [69]:
indicators['a']

[(1, 1.0), (2, 2.0)]

In [22]:
class A:
    def __init__(self):
        self._indicators = {}

    def key_in_indicators(self, key):
        if key in self._indicators:
            return True
        else:
            return False

    def get_indicator(self, key):
        if not self.key_in_indicators(key):
            return None

        return self._indicators[key]
        

    def set_indicator(self, key, time, val):
        if not self.key_in_indicators(key):
            self._indicators[key] = []
        
        self._indicators[key].append([time, val])
        
    def get_indicator_names(self):
        names = []
        for k in self._indicators:
            names.append(k)
        
        return names
       

In [23]:
a = A()

In [24]:
a.get_indicator('a')

In [26]:
a.set_indicator("b", 5, 4)

In [27]:
n = a.get_indicator_names()

In [29]:
ind = a.get_indicator('b')


In [32]:
import numpy as np
ind_n = np.array(ind)

In [33]:
ind_n

array([[4, 3],
       [5, 4]])

In [34]:
import pandas as pd

In [37]:
aa = pd.DataFrame(ind_n, columns=['timestamp', 'v'])

In [38]:
        aa['timestamp'] = pd.to_datetime((aa["timestamp"]), utc=True, unit='us')
        aa = aa.set_index('timestamp')

In [39]:
aa

,v
timestamp,
1970-01-01 00:00:00.000004+00:00,3
1970-01-01 00:00:00.000005+00:00,4
